In [1]:
from kdp import *

Let's build a graph from a region that's super simple (hom, both matching) first.

We'll have the H (haplotypes (or reads) graph) and G (graph variants to be phased/genotyped).

Starting at the snk of H - we'll have two options, use it, or don't use it.

For the use it, we check every neighboring node for G from snk and record the sizediff and cossim.
    if the sizediff and cossim is above thresholds,
        We apply Hn to Gn, update our running Hsk and Gsk (size and kmer features of the paths being joined).

if we can't use it, well.. because we have in the graph snk->everyNode, we know we've checked Use it and don't use it.
So, not using it is just saying, 'well go to the next node'.

Okay, we we have two things.
    Hi and Gi
    And we try to apply each Hi+1 to Gi+1,
    If none of them are good enough... we need to try and both Hi+1 and Hi+2 as well as just moving to Hi+2.
    That's the problem here.
    
    
```
Initialize dp[] with appropriate initial values

for each node a in graph A's full path:
    for each node b in graph B:
        if b's parent has been visited in graph A's path:
            dp[b] = max(dp[b], dp[b's parent] + weight of (b's parent to b))

Find the node b_last with the maximum value in dp[]
Backtrack from b_last to find the actual path in graph B
```

 

In [5]:
import pysam
bam = pysam.AlignmentFile("test/NA24385.chr20.bam")
ref = pysam.FastaFile("/Users/english/code/references/grch38/GRCh38_1kg_mainchrs.fa")

In [33]:
# weird deletion chrom, start, end = "chr20", 28797607-50, 28797607+120+50
chrom, start, end = "chr20", 20827970, 20827980

In [10]:
refseq = ref.fetch(chrom, start, end)
reads = bam.fetch(chrom, start, end)

In [ ]:
from truvari.phab import expand_cigar

In [11]:
reads = [_ for _ in reads]

In [12]:
def extractor(reads):
    for read in reads:
        for op, length in read.cigartuples:
            if op == 2 and length > 20:  # Deletion longer than 20 bp
                yield read.query_sequence[read.query_alignment_start:read.query_alignment_end + length]
            elif op == 1 and length > 20:  # Insertion longer than 20 bp
                yield read.query_sequence[read.query_alignment_start:read.query_alignment_end + length]

```
M	BAM_CMATCH	0
I	BAM_CINS	1
D	BAM_CDEL	2
N	BAM_CREF_SKIP	3
S	BAM_CSOFT_CLIP	4
H	BAM_CHARD_CLIP	5
P	BAM_CPAD	6
=	BAM_CEQUAL	7
X	BAM_CDIFF	8
B	BAM_CBACK	9
```

In [21]:
def cigar_positions(read, start, end):
    reference_position = read.reference_start  # Initialize reference position
    read_position = 0
    # Iterate over CIGAR tuples
    for op, length in read.cigartuples:
        if op == 5: # hard clip skip
            continue
        if op == 4: # soft clip move up
            read_position += length
            continue
        if op in (0, 8, 7): # Match
            reference_position += length
            continue
        if op 
        if op in (0, 2, 7, 8):  # Operations that consume reference positions
            yield reference_position, op, length
            if op in (0, 2):  # Match or deletion operations
                reference_position += length

In [69]:
end - start

220

In [71]:
#def extract_large_deletions(reads):
# need to track for a single read so that it has additive Ks. 
# Then I want to cluster (upto two changes) and consolidate.
# Once I have the two, then its just sending to phase.
chrom, start, end = "chr20", 28797607-50, 28797607+120+50 # deletion
all_cov = 0
for pileup_column in bam.pileup(chrom, start, end, truncate=True):
    # Check for deletions
    all_cov += pileup_column.n
    for pileup_read in pileup_column.pileups:
        if pileup_read.indel < -20:  # Deletion greater than 20 bp
            print(pileup_column.reference_pos, pileup_read.indel)
print(all_cov / (end - start))

28797606 -171
28797606 -171
28797606 -171
28797618 -170
28797618 -170
28797618 -170
28797618 -170
28797618 -170
28797618 -170
28797618 -170
28797618 -170
5449 24.76818181818182


In [72]:
#def extract_large_insertions(reads):
chrom, start, end = "chr20", 20827970, 20827980 # insertion
all_cov = 0
for pileup_column in bam.pileup(chrom, start, end, truncate=True):
    # Check for deletions
    all_cov += pileup_column.n
    for pileup_read in pileup_column.pileups:
        if pileup_read.indel > 20:  # Insertion greater than 20 bp
            print(pileup_column.reference_pos, pileup_read.indel)
            print(pileupread.alignment.query_sequence[pileupread.query_position:pileupread.query_position+pileup_read.indel])
print(all_cov / (end - start))

20827970 751
GCTCAAACAAACCCTAGGTTAAACTCTGTGTGTTGAATTCACAATTCACGAAGAAATTTATCAGAATGCTACTGTGTAGTTTTTATTTGAAGATTTTTTTTTTGCACAATCTGCATCAAAGTTCTCCAAATATCAAAAGGCAGATTCTACAAAAAGAGTGTTTCAAAACTGCTCAATCAAAAGAAAGGATTAACTCTGTGAGATGGATGCACACATCACAAAGAAGTTTCTCAGAAAGCTTCTGTGTGGTCTTTAAGTGAAGATATTTTCTTTTAAAGAGTAAGCCTCATGCGCTCCAAATATCCACTTGCAGATTCTACAAAAAGAGTGTTTCAAAACTGCTCAATCATAAAATAGGTTCAAACATGTGAGCTGAACGCACACATCACACAGAAGTTTCTCAGAATGCTTCTGTGTAGTTTTTATGTGAAGATGTTTCCTTTACCACCATAAGCCTCAAAGCACTCCAAATATCCACATGCAGATTCTACAAAAAGAGTGTTTCAAAACTGCTCAATCAAAAGAAAGATTCATCTCTGTAAGGTGAATGCACATATCACAATGTAGCTTCTGAGAAGGCTTCAGTGTACCTTTTATGTGAAGATATTTCCTTTTCCACAATGGCCTCAAACTGCTCCAAATATCCACTTGCACATTCTACAAAAAGAGTATTTCACAACTGCTCAATCATAAGATAGGTTCAACTCTGTGAGATGAATGCACACATCACAAAGAAGTTTCTCAGAATGCT
20827970 752
GCTCAAACAAACCCTAGGTTAAACTCTGTGTGTTGAATTCACAATTCACGAAGAAATTTATCAGAATGCTACTGTGTAGTTTTTATTTGAAGATTTTTTTTTTGCACAATCTGCATCAAAGTTCTCCAAATATCAAAAGGCAGATTCTACAAAAAGAGTGTTTCAAAACTGCTCAATCAAAAGAAAGGATTAACTCTGTGAGATGGATGCACACATCACAAA

In [48]:

for pileupcolumn in bam.pileup(chrom, start, end):
    if not (start <= pileupcolumn.pos <= end):
        continue
    print("\ncoverage at base %s = %s" % (pileupcolumn.pos, pileupcolumn.n))
    for pileupread in pileupcolumn.pileups:
        if not abs(pileupread.indel) > 20:
            continue
        print(pileupread.indel)
        if pileupread.indel < 0:
            print('deletion %d->%d' % (pileupcolumn.pos, pileupread.indel))
        else:
            print('insertion %d->%s' % 
                  (pileupcolumn.pos,
                   
                  ))


coverage at base 28797557 = 24

coverage at base 28797558 = 24

coverage at base 28797559 = 24

coverage at base 28797560 = 24

coverage at base 28797561 = 24

coverage at base 28797562 = 24

coverage at base 28797563 = 24

coverage at base 28797564 = 24

coverage at base 28797565 = 24

coverage at base 28797566 = 24

coverage at base 28797567 = 24

coverage at base 28797568 = 24

coverage at base 28797569 = 24

coverage at base 28797570 = 24

coverage at base 28797571 = 24

coverage at base 28797572 = 24

coverage at base 28797573 = 24

coverage at base 28797574 = 24

coverage at base 28797575 = 24

coverage at base 28797576 = 24

coverage at base 28797577 = 24

coverage at base 28797578 = 24

coverage at base 28797579 = 24

coverage at base 28797580 = 24

coverage at base 28797581 = 24

coverage at base 28797582 = 24

coverage at base 28797583 = 24

coverage at base 28797584 = 24

coverage at base 28797585 = 24

coverage at base 28797586 = 24

coverage at base 28797587 = 24

coverag

0

In [23]:
for i in cigar_positions(reads[0]):
    print(i)

(28779757, 7, 5)
(28779757, 8, 1)
(28779757, 7, 57)
(28779757, 8, 2)
(28779757, 7, 8)
(28779757, 8, 1)
(28779757, 7, 11)
(28779757, 8, 1)
(28779757, 7, 95)
(28779757, 8, 3)
(28779757, 7, 11)
(28779757, 8, 1)
(28779757, 7, 42)
(28779757, 8, 1)
(28779757, 7, 8)
(28779757, 8, 1)
(28779757, 7, 5)
(28779757, 8, 1)
(28779757, 7, 12)
(28779757, 8, 1)
(28779757, 7, 2)
(28779757, 8, 1)
(28779757, 7, 28)
(28779757, 8, 2)
(28779757, 7, 47)
(28779757, 8, 1)
(28779757, 7, 4)
(28779757, 8, 1)
(28779757, 7, 6)
(28779757, 8, 1)
(28779757, 7, 17)
(28779757, 8, 1)
(28779757, 7, 19)
(28779757, 8, 1)
(28779757, 7, 1)
(28779757, 8, 1)
(28779757, 7, 30)
(28779757, 8, 1)
(28779757, 7, 6)
(28779757, 2, 1)
(28779758, 7, 20)
(28779758, 8, 1)
(28779758, 7, 26)
(28779758, 8, 1)
(28779758, 7, 8)
(28779758, 8, 1)
(28779758, 7, 35)
(28779758, 8, 1)
(28779758, 7, 20)
(28779758, 8, 1)
(28779758, 7, 44)
(28779758, 8, 1)
(28779758, 7, 2)
(28779758, 8, 1)
(28779758, 7, 57)
(28779758, 8, 1)
(28779758, 7, 33)
(28779758, 8,